<a href="https://colab.research.google.com/github/aldojda/Processing-DataClassification/blob/main/TAREA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install unidecode
!pip install transformers

In [4]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
out = pd.read_csv("drive/MyDrive/01 - Maestría Ciencia de Datos/ProyectoSentiment-AnalysisFCFM/datos_FCFM_misprofesores_250723.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('spanish'))
stop_words.add("comentario")
stop_words.add("esperando")
stop_words.add("revisin")
stop_words.add("si")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
from unidecode import unidecode
def clean_text(text, tokenize  = False):

    cleaned_text = unidecode(text)
    # Remove special characters and symbols
    cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', cleaned_text)

    # Remove extra whitespaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    # Remove leading and trailing spaces
    cleaned_text = cleaned_text.strip()

    # Convert to lowercase
    cleaned_text = cleaned_text.lower()
    # remove stopwords & get tokens
    if tokenize == True:
        tokens = word_tokenize(cleaned_text)
        texto_filtrado = [word for word in tokens if word not in stop_words]
        return texto_filtrado
    else :
        return cleaned_text

In [65]:
import re
out.dropna(subset = ["commentario"], inplace = True)
out['clean_comments'] = out.commentario.apply(lambda x: clean_text(x, tokenize = True))
df = out.dropna(subset = ["clean_comments"])
print(df.shape)
df = df[df.clean_comments.apply(lambda x: len(x)) > 0 ].copy()
print(df.shape)
df["profesor"] = df.maestro.str.split("/").apply(\
    lambda x: x[4]).str.split("_").apply(lambda x: x[0]).str.replace("-", " ")
df["profesor"] = df["profesor"].apply(lambda x: clean_text(x))
df.drop(columns = ["maestro", "Unnamed: 9"], inplace = True)
df["materia_clean"] = df.materia.apply(lambda x: clean_text(x))
# filtramos los profesores con más de 30 comentarios
prof2select = df.profesor.value_counts()[:30].index
df = df[df.profesor.isin(prof2select)]
print(df.shape)
df.head()

(3938, 11)
(3934, 11)
(1132, 11)


,materia,obligatoria,calificacion_alumno,interes,commentario,calidad_general,facilidad,date,clean_comments,profesor,materia_clean
193,Álgebra,No obligatoria,10.0,Medio,"Su materia es 100% examen,...",6.0,10.0,02/Dic/2022,"[materia, 100, examen, pone, viene, libro, pri...",perez blanco angel,algebra
194,Álgebra Lineal,No obligatoria,10.0,Bajo,"Es una excusa de profesor,...",5.0,10.0,13/Mar/2022,"[excusa, profesor, aburridisima, clase, clase,...",perez blanco angel,algebra lineal
196,---,No obligatoria,9.0,Medio,[Comentario esperando revi...,6.0,2.0,13/Mar/2021,[revision],perez blanco angel,
197,Matemáticas Discretas,No obligatoria,10.0,Medio,"Es demasiado barco, pero i...",8.0,10.0,12/Feb/2021,"[demasiado, barco, igual, ponle, atencion, pro...",perez blanco angel,matematicas discretas
198,Matematicas Discretas,No obligatoria,10.0,Alto,Si es cierto que el maestr...,10.0,10.0,11/Ene/2021,"[cierto, maestro, saca, libro, aunque, punto, ...",perez blanco angel,matematicas discretas


In [66]:
materias2comparar = ['geometria analitica', 'calculo integral', 'algebra',
       'calculo diferencial', 'fisica basica', 'calculo de varias variables']

df = df[df.materia_clean.isin(materias2comparar)]
print(df.shape)

(349, 11)


In [41]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis',
                      model="nlptown/bert-base-multilingual-uncased-sentiment")

sentences = df.commentario.to_list()
results = classifier(sentences)


In [69]:
x = pd.DataFrame(results)
x["comment"] = sentences
#df = df.merge(x, how = "left", left_on = "commentario", right_on = "comment", validate = "1:1")
#df["label"] = df["label"].str[0].astype(int)
df["sentimiento"] = np.where(df.label<= 3, "negativo",
                             np.where(df.label == 4, "neutro", "postivo"))
df.head()

,materia,obligatoria,calificacion_alumno,interes,commentario,calidad_general,facilidad,date,clean_comments,profesor,materia_clean,label,score,comment,sentimiento
0,Álgebra,No obligatoria,10.0,Medio,"Su materia es 100% examen,...",6.0,10.0,02/Dic/2022,"[materia, 100, examen, pone, viene, libro, pri...",perez blanco angel,algebra,5,0.431279,"Su materia es 100% examen,...",postivo
1,Álgebra,No obligatoria,8.0,Bajo,"Maestro ""gracioso"" que sac...",6.0,10.0,14/Ago/2020,"[maestro, gracioso, saca, todas, clases, libro...",perez blanco angel,algebra,5,0.705543,"Maestro ""gracioso"" que sac...",postivo
2,Geometría Analítica,No obligatoria,4.0,Bajo,Da una introducción a un t...,6.0,6.0,30/Jul/2020,"[da, introduccion, tema, luego, da, clase, men...",perez blanco angel,geometria analitica,3,0.507319,Da una introducción a un t...,negativo
3,Algebra,No obligatoria,9.0,Alto,"muy buen profesor, ami me ...",7.0,8.0,07/Jun/2020,"[buen, profesor, ami, gusta, explica, 2da, vez...",perez blanco angel,algebra,4,0.616850,"muy buen profesor, ami me ...",neutro
4,Geometría Analítica,No obligatoria,9.0,Medio,"Es muy barco, haz los ejer...",7.0,6.0,08/Ene/2020,"[barco, haz, ejercicios, libro, pregunta, dudas]",perez blanco angel,geometria analitica,5,0.537116,"Es muy barco, haz los ejer...",postivo


In [75]:
print(df.materia_clean.value_counts().to_latex())

\begin{tabular}{lr}
\toprule
 & count \\
materia_clean &  \\
\midrule
geometria analitica & 73 \\
calculo integral & 72 \\
algebra & 63 \\
calculo diferencial & 54 \\
fisica basica & 46 \\
calculo de varias variables & 41 \\
\bottomrule
\end{tabular}



Se tiene 5 materias a elegir en la facultad de FCFM [

In [70]:
df.groupby("materia_clean").label.value_counts(normalize = True).unstack().style.format('{:.2%}')

label,1,2,3,4,5
materia_clean,,,,,
algebra,3.17%,9.52%,12.70%,44.44%,30.16%
calculo de varias variables,12.20%,17.07%,14.63%,34.15%,21.95%
calculo diferencial,14.81%,16.67%,24.07%,29.63%,14.81%
calculo integral,13.89%,16.67%,15.28%,22.22%,31.94%
fisica basica,17.39%,15.22%,10.87%,21.74%,34.78%
geometria analitica,21.92%,20.55%,13.70%,21.92%,21.92%


In [73]:

print(df.groupby("materia_clean").label.value_counts(normalize = True).unstack().style.format('{:.2%}').to_latex())

\begin{tabular}{lrrrrr}
label & 1 & 2 & 3 & 4 & 5 \\
materia_clean &  &  &  &  &  \\
algebra & 3.17% & 9.52% & 12.70% & 44.44% & 30.16% \\
calculo de varias variables & 12.20% & 17.07% & 14.63% & 34.15% & 21.95% \\
calculo diferencial & 14.81% & 16.67% & 24.07% & 29.63% & 14.81% \\
calculo integral & 13.89% & 16.67% & 15.28% & 22.22% & 31.94% \\
fisica basica & 17.39% & 15.22% & 10.87% & 21.74% & 34.78% \\
geometria analitica & 21.92% & 20.55% & 13.70% & 21.92% & 21.92% \\
\end{tabular}



In [71]:
df.groupby("materia_clean").sentimiento.value_counts(normalize = True).unstack().style.format('{:.2%}')

sentimiento,negativo,neutro,postivo
materia_clean,,,
algebra,25.40%,44.44%,30.16%
calculo de varias variables,43.90%,34.15%,21.95%
calculo diferencial,55.56%,29.63%,14.81%
calculo integral,45.83%,22.22%,31.94%
fisica basica,43.48%,21.74%,34.78%
geometria analitica,56.16%,21.92%,21.92%
